In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, classification_report

import lightgbm as lgbm
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import optuna

from preprocessing import agg_over_months, rolling_window
from utils import study_summary


plt.rcParams["figure.figsize"] = (12, 6)

# Setup

In [2]:
train_df = pd.read_csv("./data/train.csv", index_col=0).drop_duplicates()
X = train_df.loc[:, ~train_df.columns.isin(["LABELS"])]
y = train_df.loc[:, "LABELS"]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42, stratify=y)

In [3]:
# Constants to select pandas columns efficiently
# all months abbreviations: jan, feb, mar, etc.
MONTHS = [m.lower() for m in pd.date_range(0, freq="M", periods=12).strftime("%b").to_list()]
# return all columns based on MONTH: train_df[COL_BY_MONTH["jan"]]
COL_BY_MONTH = {}
for month in MONTHS:
    COL_BY_MONTH[month] = [col for col in X.columns if month in col]

# all features
FEATURES = [col for col in X.columns if col != "LABELS"]
# all features with the month stripped: S2_B2_
COL_BASE = list({col[:-3] for col in FEATURES})
# return all columns based on FEATURE (e.g., across months): train_df[COL_BY_FEATURE["S2_B2_"]]
# alternatively, you can select the same df using: train_df[COL_BY_FEATURE[FEATURES[0]]]
COL_BY_FEATURE = {}
for feature in COL_BASE:
    COL_BY_FEATURE[feature] = [col for col in X.columns if feature in col]

# Feature Generation

In [4]:
#generated_train_features_df = agg_over_months(X_train, agg_func=["mean", "std", "min", "max"], freq=3, append_label=False)
generated_train_features_df = rolling_window(X_train, agg_func=["mean", "std", "min", "max"], window_size=3, append_label=False)
generated_train_features_df.dropna(axis=1, how='all', inplace=True)
X_train = X_train.merge(generated_train_features_df, left_index=True, right_index=True)

In [5]:
#generated_test_features_df = agg_over_months(X_test, agg_func=["mean", "std", "min", "max"], freq=3, append_label=False)
generated_test_features_df = rolling_window(X_test, agg_func=["mean", "std", "min", "max"], window_size=3, append_label=False)
X_test = X_test.merge(generated_test_features_df, left_index=True, right_index=True)
X_test = X_test[X_train.columns]

In [6]:
print(y_train.value_counts(), y_test.value_counts())

1.0    28433
0.0    13641
Name: LABELS, dtype: int64 1.0    12186
0.0     5846
Name: LABELS, dtype: int64


# Optuna Hyperparameter Optimization

In [7]:
def objective(trial, X, y):
    # hyperparameter grid for Optuna to explore
    hyperparams = {
        # structure
        "max_depth": trial.suggest_int("max_depth", 3, 20, step=1),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 15, step=1),
        # accuracy
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "n_estimators": trial.suggest_categorical("n_estimators", [100]),
        # overfitting
        "gamma": trial.suggest_float("gamma", 0.0, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
        "subsample": trial.suggest_float("subsample", 0.0, 1.0)
    }
    
    # hold the best score of each fold
    cv_scores = []
    kfold_cv = StratifiedKFold(n_splits=5, shuffle=True)
    for idx, (train_idx, test_idx) in enumerate(kfold_cv.split(X, y)):
        # train-validation split for each fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Scale weights to deal with imblanace
        y1_scale_weights = y_train.sum() / y_train.shape[0]
        hyperparams['scale_pos_weight'] = y1_scale_weights

        
        # Train classifier with optuna hyperparameters
        clf = XGBClassifier(objective="binary:logistic", use_label_encoder=False, tree_method='gpu_hist', gpu_id=0, **hyperparams)
        clf.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric=["logloss"],
            verbose=True,
        )
        
        # store the best score to the CV score
        results = clf.evals_result()
        best_iteration = clf.best_iteration
        cv_scores.append(results['validation_0']['logloss'][best_iteration])
    
    # return the average best score across CV folds
    return np.mean(cv_scores)

In [8]:
# create optuna study
study = optuna.create_study(direction="minimize", study_name="lgbm03")
optimize = lambda trial: objective(trial, X_train, y_train)

[I 2021-12-12 14:53:48,320] A new study created in memory with name: lgbm03


In [9]:
# launch optuna study
study.optimize(optimize, n_trials=50)

[0]	validation_0-logloss:0.68995
[1]	validation_0-logloss:0.68683
[2]	validation_0-logloss:0.68402
[3]	validation_0-logloss:0.68049
[4]	validation_0-logloss:0.67667
[5]	validation_0-logloss:0.67367
[6]	validation_0-logloss:0.67053
[7]	validation_0-logloss:0.66684
[8]	validation_0-logloss:0.66339
[9]	validation_0-logloss:0.66012
[10]	validation_0-logloss:0.65759
[11]	validation_0-logloss:0.65442
[12]	validation_0-logloss:0.65129
[13]	validation_0-logloss:0.64855
[14]	validation_0-logloss:0.64548
[15]	validation_0-logloss:0.64356
[16]	validation_0-logloss:0.64085
[17]	validation_0-logloss:0.63812
[18]	validation_0-logloss:0.63586
[19]	validation_0-logloss:0.63294
[20]	validation_0-logloss:0.63049
[21]	validation_0-logloss:0.62830
[22]	validation_0-logloss:0.62548
[23]	validation_0-logloss:0.62322
[24]	validation_0-logloss:0.62092
[25]	validation_0-logloss:0.61847
[26]	validation_0-logloss:0.61543
[27]	validation_0-logloss:0.61315
[28]	validation_0-logloss:0.61037
[29]	validation_0-loglos

[I 2021-12-12 14:54:31,005] Trial 0 finished with value: 0.506289 and parameters: {'max_depth': 17, 'min_child_weight': 1, 'learning_rate': 0.012511324976413612, 'n_estimators': 100, 'gamma': 0.2317262406008478, 'reg_alpha': 0.5969330731140412, 'reg_lambda': 0.42589193133105907, 'subsample': 0.010688557168244128}. Best is trial 0 with value: 0.506289.


[0]	validation_0-logloss:0.59184
[1]	validation_0-logloss:0.53116
[2]	validation_0-logloss:0.48899
[3]	validation_0-logloss:0.45969
[4]	validation_0-logloss:0.43714
[5]	validation_0-logloss:0.41909
[6]	validation_0-logloss:0.40949
[7]	validation_0-logloss:0.39920
[8]	validation_0-logloss:0.39193
[9]	validation_0-logloss:0.38767
[10]	validation_0-logloss:0.38212
[11]	validation_0-logloss:0.38017
[12]	validation_0-logloss:0.37877
[13]	validation_0-logloss:0.37785
[14]	validation_0-logloss:0.37575
[15]	validation_0-logloss:0.37489
[16]	validation_0-logloss:0.37476
[17]	validation_0-logloss:0.37614
[18]	validation_0-logloss:0.37668
[19]	validation_0-logloss:0.37762
[20]	validation_0-logloss:0.37704
[21]	validation_0-logloss:0.37761
[22]	validation_0-logloss:0.37710
[23]	validation_0-logloss:0.37802
[24]	validation_0-logloss:0.37791
[25]	validation_0-logloss:0.37869
[26]	validation_0-logloss:0.37907
[27]	validation_0-logloss:0.37904
[28]	validation_0-logloss:0.38026
[29]	validation_0-loglos

[I 2021-12-12 14:57:44,073] Trial 1 finished with value: 0.40864540000000005 and parameters: {'max_depth': 18, 'min_child_weight': 5, 'learning_rate': 0.25907290935840016, 'n_estimators': 100, 'gamma': 0.9257145190293854, 'reg_alpha': 0.6798413034633856, 'reg_lambda': 0.30638953273870273, 'subsample': 0.5524529881256859}. Best is trial 1 with value: 0.40864540000000005.


[0]	validation_0-logloss:0.66745
[1]	validation_0-logloss:0.64403
[2]	validation_0-logloss:0.62198
[3]	validation_0-logloss:0.60400
[4]	validation_0-logloss:0.58754
[5]	validation_0-logloss:0.57290
[6]	validation_0-logloss:0.56007
[7]	validation_0-logloss:0.54852
[8]	validation_0-logloss:0.53731
[9]	validation_0-logloss:0.52712
[10]	validation_0-logloss:0.51610
[11]	validation_0-logloss:0.50740
[12]	validation_0-logloss:0.49904
[13]	validation_0-logloss:0.49131
[14]	validation_0-logloss:0.48337
[15]	validation_0-logloss:0.47756
[16]	validation_0-logloss:0.47112
[17]	validation_0-logloss:0.46447
[18]	validation_0-logloss:0.45974
[19]	validation_0-logloss:0.45481
[20]	validation_0-logloss:0.45074
[21]	validation_0-logloss:0.44701
[22]	validation_0-logloss:0.44276
[23]	validation_0-logloss:0.43891
[24]	validation_0-logloss:0.43508
[25]	validation_0-logloss:0.43233
[26]	validation_0-logloss:0.42932
[27]	validation_0-logloss:0.42664
[28]	validation_0-logloss:0.42337
[29]	validation_0-loglos

...

[I 2021-12-12 18:45:17,509] Trial 47 finished with value: 0.3475374 and parameters: {'max_depth': 16, 'min_child_weight': 9, 'learning_rate': 0.033170821575077815, 'n_estimators': 100, 'gamma': 0.3560418282617922, 'reg_alpha': 0.09492938155018837, 'reg_lambda': 0.2742416487731015, 'subsample': 0.9701633531894849}. Best is trial 32 with value: 0.33506179999999997.


[0]	validation_0-logloss:0.63875
[1]	validation_0-logloss:0.59552
[2]	validation_0-logloss:0.55915
[3]	validation_0-logloss:0.53041
[4]	validation_0-logloss:0.50699
[5]	validation_0-logloss:0.48749
[6]	validation_0-logloss:0.47051
[7]	validation_0-logloss:0.45562
[8]	validation_0-logloss:0.44369
[9]	validation_0-logloss:0.43315
[10]	validation_0-logloss:0.42420
[11]	validation_0-logloss:0.41643
[12]	validation_0-logloss:0.40955
[13]	validation_0-logloss:0.40319
[14]	validation_0-logloss:0.39790
[15]	validation_0-logloss:0.39300
[16]	validation_0-logloss:0.38910
[17]	validation_0-logloss:0.38522
[18]	validation_0-logloss:0.38197
[19]	validation_0-logloss:0.37937
[20]	validation_0-logloss:0.37659
[21]	validation_0-logloss:0.37408
[22]	validation_0-logloss:0.37148
[23]	validation_0-logloss:0.37005
[24]	validation_0-logloss:0.36906
[25]	validation_0-logloss:0.36764
[26]	validation_0-logloss:0.36657
[27]	validation_0-logloss:0.36559
[28]	validation_0-logloss:0.36438
[29]	validation_0-loglos

[I 2021-12-12 18:47:46,894] Trial 48 finished with value: 0.3473122 and parameters: {'max_depth': 10, 'min_child_weight': 7, 'learning_rate': 0.13160297964966053, 'n_estimators': 100, 'gamma': 0.8695144644211031, 'reg_alpha': 0.17796793312296277, 'reg_lambda': 0.4416275917615085, 'subsample': 0.6441617118086322}. Best is trial 32 with value: 0.33506179999999997.


[0]	validation_0-logloss:0.64529
[1]	validation_0-logloss:0.60502
[2]	validation_0-logloss:0.57171
[3]	validation_0-logloss:0.54448
[4]	validation_0-logloss:0.52096
[5]	validation_0-logloss:0.50004
[6]	validation_0-logloss:0.48190
[7]	validation_0-logloss:0.46593
[8]	validation_0-logloss:0.45211
[9]	validation_0-logloss:0.44033
[10]	validation_0-logloss:0.42961
[11]	validation_0-logloss:0.41941
[12]	validation_0-logloss:0.41138
[13]	validation_0-logloss:0.40409
[14]	validation_0-logloss:0.39698
[15]	validation_0-logloss:0.39081
[16]	validation_0-logloss:0.38592
[17]	validation_0-logloss:0.38099
[18]	validation_0-logloss:0.37688
[19]	validation_0-logloss:0.37309
[20]	validation_0-logloss:0.36944
[21]	validation_0-logloss:0.36669
[22]	validation_0-logloss:0.36402
[23]	validation_0-logloss:0.36090
[24]	validation_0-logloss:0.35892
[25]	validation_0-logloss:0.35706
[26]	validation_0-logloss:0.35513
[27]	validation_0-logloss:0.35344
[28]	validation_0-logloss:0.35226
[29]	validation_0-loglos

[I 2021-12-12 18:54:26,333] Trial 49 finished with value: 0.3455016 and parameters: {'max_depth': 19, 'min_child_weight': 5, 'learning_rate': 0.10719932475947233, 'n_estimators': 100, 'gamma': 0.6255260548786011, 'reg_alpha': 0.3661497830903253, 'reg_lambda': 0.6511029306612424, 'subsample': 0.8896043272973921}. Best is trial 32 with value: 0.33506179999999997.


In [10]:
study_summary(study)

Study:  lgbm03

Number of finished trials: 50
Best trial:
  Validation score: 0.33506179999999997
  Params: 
    max_depth: 19
    min_child_weight: 7
    learning_rate: 0.07177069561151314
    n_estimators: 100
    gamma: 0.44725306980437746
    reg_alpha: 0.4503611808519714
    reg_lambda: 0.3500218826498879
    subsample: 0.9350968745387884


In [11]:
X_df = X_train.copy()
y_df = y_train.copy()
X_test_df = X_test.copy()
y_test_df = y_test.copy()

In [12]:
def train_test_model(X_train,y_train,X_test,y_test, study):
    clf = XGBClassifier(objective="binary:logistic",
                        scale_pos_weight=y_train.sum()/y_train.shape[0],
                        use_label_encoder=False,
                        tree_method='gpu_hist',
                        gpu_id=0,
                        **study.best_params)
    clf = clf.fit(
            X_train,
            y_train.astype(int),
            eval_metric=["logloss"],
            verbose=True,
        )
    
    y_pred = clf.predict(X_test).astype(int)
    y_prob = clf.predict_proba(X_test)

    return (clf, y_pred, y_prob, y_test)

In [13]:
(clf, y_pred, y_prob, y_test) = train_test_model(X_df, y_df, X_test_df, y_test_df, study)
print(classification_report(y_test, y_pred))
f1_score(y_test, y_pred)

              precision    recall  f1-score   support

         0.0       0.76      0.76      0.76      5846
         1.0       0.89      0.88      0.88     12186

    accuracy                           0.84     18032
   macro avg       0.82      0.82      0.82     18032
weighted avg       0.84      0.84      0.84     18032



0.8841533651472766

In [14]:
features = pd.DataFrame()
features['columns'] = X_df.columns
features['importances'] = clf.feature_importances_
features.sort_values(by='importances', ascending=True, inplace=True)

In [15]:
features_reduced = features[features['importances'] > 0.001]['columns'].tolist()

In [16]:
len(features_reduced)

265

In [17]:
(clf, y_pred, y_prob, y_test) = train_test_model(X_df[features_reduced], y_df, X_test_df[features_reduced], y_test_df, study)
print(classification_report(y_test, y_pred))
f1_score(y_test, y_pred)

              precision    recall  f1-score   support

         0.0       0.76      0.78      0.77      5846
         1.0       0.89      0.88      0.89     12186

    accuracy                           0.85     18032
   macro avg       0.83      0.83      0.83     18032
weighted avg       0.85      0.85      0.85     18032



0.8866507058532156

In [18]:
# Train on full data
#generated_X_features_df = agg_over_months(X, agg_func=["mean", "std", "min", "max"], freq=3, append_label=False)
generated_X_features_df = rolling_window(X, agg_func=["mean", "std", "min", "max"], window_size=3, append_label=False)
generated_X_features_df.dropna(axis=1, how='all', inplace=True)
X_full_train = X.merge(generated_X_features_df, left_index=True, right_index=True)

In [19]:
clf = XGBClassifier(objective="binary:logistic",
                        scale_pos_weight=y.sum()/y.shape[0],
                        use_label_encoder=False,
                        tree_method='gpu_hist',
                        gpu_id=0,
                        **study.best_params)
clf = clf.fit(
            X_full_train[features_reduced],
            y.astype(int),
            eval_metric=["logloss"],
            verbose=True,
        )
    

In [20]:
output = {
    "hyperparams": study.best_params,
    "features": features_reduced
}


In [21]:
held_out_test = pd.read_csv("data/test_nolabels.csv")
#generated_held_out_test_features_df = agg_over_months(held_out_test, agg_func=["mean", "std", "min", "max"], freq=3, append_label=False)
generated_held_out_test_features_df = rolling_window(held_out_test, agg_func=["mean", "std", "min", "max"], window_size=3, append_label=False)
X_held_out = held_out_test.merge(generated_held_out_test_features_df, left_index=True, right_index=True)
X_held_out = X_held_out[features_reduced]

In [22]:
y_pred = clf.predict(X_held_out)
y_prob = clf.predict_proba(X_held_out)

In [23]:
submission = pd.DataFrame()
submission["S.No"] = held_out_test["S.No"]
submission["LABELS"] = y_pred

In [24]:
submission.to_csv("submissions/submission8.csv", index=False)

In [25]:
import pickle

In [26]:
with open("submissions/submission8_params.pkl", "wb") as f:
    pickle.dump(output, f)

In [27]:
with open("submissions/submission8_model.pkl", "wb") as f:
    pickle.dump(clf, f)

In [30]:
features_reduced

['NDVI_jun',
 'S1_VH_jul',
 'S2_B5_sep_roll_3_std',
 'S1_VV_sep_roll_3_mean',
 'ERA5_total_precipitation_jul_roll_3_std',
 'S2_B8_mar_roll_3_max',
 'S2_B4_apr_roll_3_mean',
 'S2_B2_dec_roll_3_max',
 'S1_VH_jun_roll_3_max',
 'S2_B4_jan',
 'S1_VH_apr',
 'S2_B8_dec',
 'S2_B3_mar',
 'NDVI_may_roll_3_std',
 'S2_B12_jun_roll_3_mean',
 'ERA5_total_precipitation_nov_roll_3_mean',
 'S2_B7_sep_roll_3_mean',
 'S1_VV_apr_roll_3_max',
 'S1_VH_jul_roll_3_min',
 'NDVI_mar',
 'S2_B2_jun_roll_3_max',
 'S2_B6_aug_roll_3_min',
 'ERA5_total_precipitation_nov',
 'S2_B9_nov_roll_3_min',
 'S2_B5_jun_roll_3_max',
 'S2_B9_dec_roll_3_max',
 'S2_B7_mar_roll_3_mean',
 'S1_VV_jan',
 'S2_B4_nov_roll_3_min',
 'S1_VH_aug_roll_3_min',
 'NDVI_may_roll_3_min',
 'S2_B8_sep',
 'S1_VH_oct_roll_3_max',
 'S2_B3_sep_roll_3_min',
 'ERA5_total_precipitation_mar_roll_3_mean',
 'S2_B8A_dec_roll_3_mean',
 'ERA5_total_precipitation_may_roll_3_min',
 'S2_B6_oct_roll_3_mean',
 'S2_B8_jul_roll_3_mean',
 'NDVI_aug_roll_3_mean',
 'NDVI_